<img src="logo.png">

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
datos = pd.read_csv("datos_pipelines.csv")

In [ ]:
datos.head()

In [ ]:
from sklearn import preprocessing, feature_extraction

Como vimos en un apartado anterior, en este ejemplo vamos a modificar cada variable en función de su tipo. Al conjunto de pasos que siguen los datos se le llama comúnmente **Pipelines** (literalmente, sistemas de tuberias).

<img src="ml19.png">

**Inicio**

vamos a modificar dos transformadores de scikitlearn para que sean compatibles con pipelines. Este paso es necesario en la version actual de scikit-learn, pero seguramente será arreglado en el futuro

In [ ]:
##### Modificar transformadores

class BinarizadorCategorico(preprocessing.LabelBinarizer):
    def fit(self, X, y=None):
        super(BinarizadorCategorico, self).fit(X)
        
    def transform(self, X, y=None):
        return super(BinarizadorCategorico, self).transform(X)

    def fit_transform(self, X, y=None):
        return super(BinarizadorCategorico, self).fit(X).transform(X)
    
    
class CodificadorCategorico(preprocessing.LabelEncoder):
    def fit(self, X, y=None):
        super(CodificadorCategorico, self).fit(X)
        
    def transform(self, X, y=None):
        return super(CodificadorCategorico, self).transform(X)

    def fit_transform(self, X, y=None):
        return super(CodificadorCategorico, self).fit(X).transform(X)    

En primer lugar vamos a definir los transformadores de forma similar a como hicimos la última vez, solo que en vez de usar `OneHotEncoder` vamos a usar nuestra version de sklearn `LabelBinarizer` que hace la codificación one hot directamente sobre una variable categórica.

In [ ]:
# Cambiar nombres de las columnas

from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

col_numericas =  ['col_inexistente1', 'col2', 'col3', 'col_outliers', 'col_outliers2']
col_categorica = ['col_categorica']
col_texto = ['col_texto']
col_ordinal = ['col_ordinal']

imputador = SimpleImputer(missing_values=np.nan, copy=False)
escalador = preprocessing.StandardScaler()

transformador_ordinal = CodificadorCategorico()
transformador_categorico = BinarizadorCategorico()

transformador_texto = feature_extraction.text.TfidfVectorizer()

estimador = LogisticRegression()

Vemos que con el Binarizador transformamos como:

In [ ]:
datos[col_categorica]

In [ ]:
transformador_categorico.fit_transform(datos[col_categorica])

Que es mucho más sencillo que cómo lo hicimos la vez anterior:

In [ ]:
preprocessing.OneHotEncoder().fit_transform(
    transformador_ordinal.fit_transform(datos[col_categorica]).reshape(1000,1)
).toarray()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.pipeline import Pipeline

Un pipeline de sklearn se define como una secuencia de pasos. Cada paso se define con una tupla de forma `(nombre del paso, transformador)`

Por ejemplo, si queremos crear un pipeline que procese las variables numéricas, primero imputándolas y después estandarizandolas, podriamos crear un pipeline como:

In [ ]:
########
transformador_numerico = Pipeline(
     [('imputador', imputador), ('escalador', escalador)]
)

In [ ]:
transformador_numerico

Ahora tenemos definidos los pasos que queremos aplicar a cada variable.

In [ ]:
transformador_numerico.fit_transform(datos[col_numericas])

Pero seguimos teniendo el mismo problema de siempre, como podemos aplicar determinados transformadores a determinadas variables?

Bien, para los casos en los que tenemos un dataframe de Pandas, una opcion es crear un transformador customizado que simplemente selecciones columnas de un dataframe.

En `scikit-learn` podemos crear nuestros propios transformadores creando una clase que herede de `TransformerMixin` y que tenga el mètodo `transform`.

In [ ]:
from sklearn.base import TransformerMixin

class TransformadorBase(TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X

Vamos a crear dos transformadores, un `DenseTransformer` que convierte una matriz `sparse` en un array (tomado de [mlxtend](http://rasbt.github.io/mlxtend/), y `ColumnExtractor` que devuelve una selección de columnas. 

In [ ]:
########
from sklearn.base import BaseEstimator
from scipy.sparse import issparse


# http://rasbt.github.io/mlxtend/
class DenseTransformer(BaseEstimator):
    def __init__(self, return_copy=True):
        self.return_copy = return_copy
        self.is_fitted = False

    def transform(self, X, y=None):
        if issparse(X):
            return X.toarray()
        elif self.return_copy:
            return X.copy()
        else:
            return X

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

class ColumnExtractor(TransformerMixin):

    def __init__(self, columns):
        self.columns = columns
        
    def transform(self, X, **transform_params):
        return X[self.columns].to_numpy()
    def fit(self, X, y=None, **fit_params):
        return self

Por ejemplo si creamos un ColumnExtractor pasandole las columnas numéricas tenemos un transformador que podemos usar en un pipeline y que simplemente selecciona un subgrupo de columnas (devolviendolas como matriz)

In [ ]:
cext = ColumnExtractor(columns=col_numericas)

In [ ]:
cext.fit_transform(datos)

Creamos ahora los pipelines para cada tipo de variable. En algunos casos he añadido pasos adicionales por dos motivos. El primero, que determinados elementos de sklearn esperan datos ligeramente distintos. En segundo lugar, para conseguir que la salida de cada pipeline tenga la misma forma (un array de arrays).

In [ ]:
########

pipeline_numerico = Pipeline([
    ['selector_numerico', ColumnExtractor(columns=col_numericas)],
    ['transformador_numerico', transformador_numerico]
])

pipeline_numerico.fit_transform(datos)[:5]

In [ ]:
########

def mi_funcion1(x):
    return x[:,0]

pipeline_texto = Pipeline([
        ['selector_texto', ColumnExtractor(columns=col_texto)],
        ['transformador_dim', preprocessing.FunctionTransformer(mi_funcion1, validate=False)],
        ['transformador_texto', transformador_texto],
        ['texto_array', DenseTransformer()]
    ])

pipeline_texto.fit_transform(datos)[:5]

In [ ]:
########

pipeline_categorico = Pipeline([
    ['selector_categorica', ColumnExtractor(columns=col_categorica)],
    ['transformador_categorico', transformador_categorico]
])

pipeline_categorico.fit_transform(datos)[:5]

En el caso del pipeline ordinal hay que manipular las dimensiones de los arrays dado que trabaja con un vector de dimension 1.

In [ ]:
########

def mi_funcion2(x):
    return np.vstack(x[:])

pipeline_ordinal = Pipeline([
    ['selector_ordinal', ColumnExtractor(columns=col_ordinal)],
    ['transformador_dim1', preprocessing.FunctionTransformer(mi_funcion1, validate=False)],
    ['transformador_ordinal', transformador_ordinal],
    ['transformador_dim2', preprocessing.FunctionTransformer(mi_funcion2, validate=False)],
])


pipeline_ordinal.fit_transform(datos)[:5]

Ya tenemos una manera de, dado un conjunto de datos, separarlos y aplicar distintas transformaciones a cada variable. Nos falta una manera de, una vez se han transformado, reunirlas de nuevo.

Para ello podemos usar `FeatureUnion`, que simplemente toma un conjunto de pasos de un pipeline y los une.

In [ ]:
from sklearn.pipeline import FeatureUnion

In [ ]:
########

pipeline_procesado = FeatureUnion([
    ('variables_numericas', pipeline_numerico),
    ('variables_ordinales', pipeline_ordinal),
    ('variables_texto', pipeline_texto),
    ('variables_categoricas', pipeline_categorico),
])

In [ ]:
pipeline_procesado

In [ ]:
pipeline_procesado.fit_transform(datos)

Finalmente, necesitamos añadir un estimador al final para predecir con base a los datos transformados

In [ ]:
########

pipeline_estimador = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', estimador)
])

In [ ]:
########
pipeline_estimador.fit(datos,datos["objetivo"])

In [ ]:
pipeline_estimador.predict(datos)[:5]

In [ ]:
datos["objetivo"]

El beneficio de los pipelines, no solo es tener codigo mas legible y poder gestionar de forma ordenada todo el ciclo de vida del modelado, sino que los pipelines tienen todos los beneficios de los objetos de scikitlearn, por ejemplo, podemos usar validacion cruzada directamente con el pipeline.

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(pipeline_estimador, X=datos.drop('objetivo', axis=1), y=datos["objetivo"], scoring='roc_auc', cv=5).mean()

In [ ]:
pipeline_estimador.get_params().keys()

In [ ]:
import pickle
with open("modelo_final.pickle", "wb") as file:
    pickle.dump(pipeline_estimador, file)

In [ ]:
import pickle
with open('modelo_final.pickle', "rb") as file:
    mi_modelo = pickle.load(file)

In [ ]:
mi_modelo.predict(datos[0:5])